## VeloBall Demonstration

I'll use this notebook to demonstrate the application of VeloBall to help inform professional cycling teams about the expected payoffs associated with sending a rider with a certain profile to a specific race.

### A brief note on context
Cycling isn't popular compared the top sports, but in Europe, professional cycling teams across multiple divisions are generally funded by a myriad of sponsors who seek to gain exposure and attention through success from the team they sponsor. There are a number of ways that a team/rider can "succeed" in cycling races. A rider can place well on a given day. One-off cycling results are extremely difficult to predict, as the specifics of a given course and in-race tactics (in addition to luck) can significantly influence race results. The other measure of "success" is the general classification (GC). GC is specific to "stage races," which are effectively races contested over multiple individual races (stages), each of which with its own winner. The rider with the smallest aggregate time across all the stages of the race is the winner of that race's GC. The Tour de France is a stage race with 21 stages. When people discuss the winner of the Tour de France, they are referring to the winner of the GC of the Tour de France.

As I mentioned above, sponsors of cycling teams are seeking exposure to the general public through the results of their teams. However, these teams have limited rosters and certain riders are only capable of certain things. Spefically, many teams find themselves sending riders to races seeking top GC results when this may not be realistic. Sending a rider to a race such as the Tour de France brings with it a significant opportunity cost, as contesting the GC of a race such as the Tour de France is a 21-day commitment for the rider, and significantly hampers their ability to attempt to win individual stages of the race, or to have success at other races which precede or postcede the Tour. 

There are a number of ways one can measure the overall performance of a cycling team over the course of a system. One of these is UCI Points, the official scoring system of cycling's international governing body. Professional road cycling recently introduced a promotion/relegation system (like that of pro soccer), with important ramifications for teams with regards to race invites and sponsor dollars. UCI Points govern the promotion and relegation of teams, and thus are extremely important. <b> Thus, this first iteration of VeloBall will focus on providing a team with an expected value of points scored from GC by a rider from one of cycling's three 3-week grand tours, which offer the largest amounts of UCI Points for GC, but also are the most hotly contested races. This will help a team determine if it is "worth it" for one of their riders to contest GC at that grand tour, or if they should focus on other things. Such a determination is something teams often struggle with. </b>

In [65]:
# ===== Imports ===== #
import pandas as pd

### Training Data
I give a more detailed description of the training data in my report, but I load the training data here as a reference for the user. The training data include rider characteristics such as their previous performances in grand tours, and two different Elo rating metrics (briefly described below), which encode information about the rider's general GC riding ability. Further, the training data also included relative features about a rider's competition, such as the difference between a rider's general Elo rating and the general GC Elo rating of the top-rated rider entered in the race. <b> These relative features (1_diff, 3_diff, 5_diff, 10_diff) were some of the most important features for VeloBall's Watson AutoAI XGBoost model, with a rider's general GC Elo rating also being one of the most important. </b>

#### Elo Ratings
The generalized GC Elo rating ("elo_rating" column) and the TT Elo rating ("tt_elo_rating" column) are metrics I developed myself outside of this class. I used them for my VeloBall project as I thought they'd make informative features for a model such as the one I'm presenting here. Elo Ratings were originally devised as a rating system for chess (see here: https://en.wikipedia.org/wiki/Elo_rating_system#:~:text=The%20Elo%20rating%20system%20is,a%20Hungarian%2DAmerican%20physics%20professor), but I altered the algorithm so that it worked for a multiplayer game such as cycling.

In [69]:
# ===== Training data ===== #
# Here, each sample corresponds to a rider who entered one of cycling's 3 grand tours.
# The target column is the "actual" feature, which encodes the result of the rider in
# that grand tour. My VeloBall model is a multiclass classifier, with class 0 encoding
# victory, class 1 encoding second place, etc. Class 9 therefore encodes 10th place, and
# class 10 encompasses any result outside of the top 10.
train_df = pd.read_csv('veloball-demonstration-training.csv')
print(f'Number of training examples: {len(train_df.index)}')
train_df.head(10)

Number of training examples: 8960


,elo_ranking,elo_rating,tt_elo_ranking,tt_elo_rating,1_diff,3_diff,5_diff,10_diff,tt_1_diff,tt_3_diff,tt_5_diff,tt_10_diff,tt_kms,grand_tour_wins,grand_tour_podiums,grand_tour_top10s,actual
0,18,1562.952640,29,1519.260125,135.815804,109.047382,96.591855,40.455111,192.362656,124.855071,99.656768,36.706587,89.3,0,0,0,10
1,47,1499.848839,80,1494.635758,385.851873,136.632172,114.739055,84.952918,140.212232,108.097086,104.088800,73.520876,60.5,0,0,0,10
2,14,1558.410014,101,1500.000000,156.713437,140.880633,116.630695,39.947812,315.731768,156.795108,89.241343,47.020819,46.0,0,0,0,10
3,70,1502.364515,99,1499.559759,286.366431,225.713534,193.485194,118.291762,208.448729,184.045917,99.605830,76.977192,101.4,0,0,0,10
4,93,1490.489867,101,1500.000000,161.998933,136.370032,91.929046,67.731755,174.664936,114.507962,66.161256,46.792627,64.9,0,0,0,10
5,75,1495.161719,24,1502.389840,145.445618,81.090014,48.505065,32.800092,114.222469,32.951416,27.697494,15.875456,58.9,0,0,0,10
6,60,1506.532718,67,1510.236141,312.726711,125.333951,116.860793,92.231125,147.362348,108.664087,103.948122,63.636546,74.0,0,0,0,10
7,41,1508.116403,101,1500.000000,182.827066,120.659260,100.741388,64.407955,145.403818,60.381976,53.313543,43.969880,34.9,0,0,0,10
8,97,1494.366985,97,1497.081646,287.260481,162.907084,139.099548,87.089702,252.560238,130.442890,112.450602,60.014417,47.0,0,0,0,10
9,16,1535.326689,37,1504.006266,122.184530,65.292932,54.627596,18.154489,171.603525,80.953325,66.215360,47.265791,46.3,0,0,0,7


## Using the VeloBall Model
I unfortunately ran out of free Watson ML compute resources during this project, and thus can't demonstrate a running of the VeloBall model using Watson's Python library. Instead, I will load the result of a dataset of 150 rider profiles I fed to the model as a batch and demonstrate how VeloBall would be used, using that model output.

In [58]:
 wml_credentials = {
    #"instance_id": "wml_local",
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey": "VYJYtrDqvL7T3GuY36diX4rQYjfTvHkfpmsa6qxKE1Z3",
    "version": "2.0"
}
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [24]:
from ibm_watson_machine_learning.experiment import AutoAI

experiment = AutoAI(wml_credentials,
    project_id='293f42cc-ba7a-4740-a413-84480cccb797')

pipeline_optimizer = experiment.optimizer(
            name='VeloGC-2',
            prediction_type=AutoAI.PredictionType.MULTICLASS,
            prediction_column='actual',
            scoring=AutoAI.Metrics.PRECISION_SCORE,
            holdout_size=0.1,
            #max_num_daub_ensembles=1,
            train_sample_rows_test_size=1.,
#             daub_include_only_estimators = [
#                  AutoAI.ClassificationAlgorithms.XGB,
#                  AutoAI.ClassificationAlgorithms.LGBM
#                  ],
#             cognito_transform_names = [
#                  AutoAI.Transformers.SUM,
#                  AutoAI.Transformers.MAX
#                  ]
        )

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [35]:
batch = Batch(
    wml_credentials, 
    space_id = 'a1e212d6-8498-4c3f-9dfc-0049a337f496', 
    source_project_id = '293f42cc-ba7a-4740-a413-84480cccb797'
)
batch.get(deployment_id = '7f0abb8f-8b53-4025-a18b-37db84ce7874')

/Users/samuelhmorton/opt/anaconda3/lib/python3.7/site-packages/ibm_watson_machine_learning/deployment/base_deployment.py:81: UserWarning:

"space_id" parameter is deprecated, please use "source_space_id"



"space_id" parameter is deprecated, please use "source_space_id"
Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.
Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [38]:
res = batch.run_job(payload = test_X_df)

In [60]:
jobs = batch.list_jobs()
jobs

,job id,state,creted,deployment id
0,1275841a-95de-492a-82c2-e7f86e0c10ec,failed,2022-05-30T20:03:52.485Z,7f0abb8f-8b53-4025-a18b-37db84ce7874
1,e4256c43-59ff-4356-8aed-bc8ecbbfeb6a,failed,2022-05-30T20:03:30.373Z,7f0abb8f-8b53-4025-a18b-37db84ce7874
2,e9f0f49b-3963-4697-872e-0009fc150bec,completed,2022-05-25T20:24:21.164Z,7f0abb8f-8b53-4025-a18b-37db84ce7874


In [63]:
batch.get_job_result(scoring_job_id = 'e9f0f49b-3963-4697-872e-0009fc150bec')

KeyboardInterrupt: 

In [59]:
batch

name: VeloGC XG Boost With Rider Info, id: 7f0abb8f-8b53-4025-a18b-37db84ce7874, asset_id: None